# Basic Example: Train a MOFA model

This notebook is a good introduction how to use PRISMO by training a MOFA model [1] with additional sparsity priors. We use a chronic lymphocytic leukaemia (CLL) data set, which combines ex vivo drug response measurements with somatic mutation status, transcriptome profiling and DNA methylation assays [2].

We reproduce results from the paper [1] and explain in detail how you can extend the model and generate meaningful interpretations of the results.

[1] Multi-Omics Factor Analysis-a framework for unsupervised integration of multi-omics data sets by Argelaguet, R. et al. (2018)  
[2] Drug-perturbation-based stratification of blood cancer by Dietrich et al. (2018)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
from data_loader import load_CLL

from famo.core import CORE
from famo.plotting import (
    plot_factor,
    plot_factor_correlation,
    plot_top_weights,
    plot_training_curve,
    plot_variance_explained,
    plot_weights,
)

## Load and Preprocess Data

In [3]:
# Load CLL data
mdata = load_CLL()

# Normalize and log transform mRNA counts
sc.pp.normalize_total(mdata["mrna"], target_sum=1e4)
sc.pp.log1p(mdata["mrna"])


## Example 1: Train a Factor Analysis Model

In the first experiment, we will use all four views and train a factor analysis model. Each model training consists of two necessary steps:

1) Create a new model instance: `model = CORE()`
2) Train the model: `model.fit(data=mdata, n_factors=15)`

To simplify your life, we only require you to specify a number of factors and provide the training data. In Example 2, we will show you how to deviate from the default parameters and customize your model. For simplicity, we will model all data using a Gaussian noise model.
In contrast to the experiments shown in the original MOFA paper, we place a Horseshoe sparsity prior [3] on the weights due to its superior computational properties over the Spike-and-Slab prior.

[3] Carvalho, Carlos M., Nicholas G. Polson, and James G. Scott. "Handling sparsity via the horseshoe." Artificial intelligence and statistics. PMLR, 2009.

In [4]:
import altair as alt
import pandas as pd
import numpy as np

# Create the array of values from 0 to 4
values = np.arange(5)

# Create a DataFrame for the heatmap
df = pd.DataFrame({
    'x': range(5),
    'y': [0] * 5,
    'value': values
})

# Create the heatmap using Altair
heatmap = alt.Chart(df).mark_rect().encode(
    x='x:O',
    y='y:O',
    color=alt.Color('value:Q', scale=alt.Scale(scheme='redblue'))
).properties(
    width=300,
    height=50,
    title='Heatmap with Redblue Colormap'
)

# Display the chart
heatmap


alt.Chart(...)

In [5]:
model = CORE(device="cuda")
model.fit(
    data=mdata,                # Our training data
    n_factors=20,              # number of factors              
    likelihoods="Normal",      # Likelihood for all views
    factor_prior="Normal",
    weight_prior="Horseshoe",  # Sparsity prior for the weights
    lr=0.005,
    early_stopper_patience=500,
)

2024-10-15 09:52:35 | famo.core | Setting up device...
2024-10-15 09:52:36 | famo.core | - No device id given. Using default device: 0
2024-10-15 09:52:36 | famo.core | - Running all computations on `cuda:0`
2024-10-15 09:52:36 | famo.core | - Using provided likelihood for all views.
2024-10-15 09:52:36 | famo.core |   - drugs: Normal
2024-10-15 09:52:36 | famo.core |   - methylation: Normal
2024-10-15 09:52:36 | famo.core |   - mrna: Normal
2024-10-15 09:52:36 | famo.core |   - mutations: Normal


alt.Chart(...)

2024-10-15 09:52:36 | famo.core | Initializing factors using `random` method...
2024-10-15 09:52:37 | famo.core | Decaying learning rate over 10000 iterations.
2024-10-15 09:52:37 | famo.core | Setting training seed to `2410150952`.
2024-10-15 09:52:37 | famo.core | Cleaning parameter store.
2024-10-15 09:52:38 | root | Guessed max_plate_nesting = 3
2024-10-15 09:52:38 | famo.core | Epoch:       0 | Time:       1.38s | Loss:   13749.82
2024-10-15 09:52:43 | famo.core | Epoch:     100 | Time:       5.84s | Loss:   10500.63
2024-10-15 09:52:47 | famo.core | Epoch:     200 | Time:      10.41s | Loss:    8924.30
2024-10-15 09:52:52 | famo.core | Epoch:     300 | Time:      14.86s | Loss:    8126.24
2024-10-15 09:52:56 | famo.core | Epoch:     400 | Time:      19.31s | Loss:    7599.94
2024-10-15 09:53:01 | famo.core | Epoch:     500 | Time:      23.77s | Loss:    7270.99
2024-10-15 09:53:05 | famo.core | Epoch:     600 | Time:      28.22s | Loss:    7074.01
2024-10-15 09:53:10 | famo.core 

Looking at the logs, we can see the following:
- for computational efficiency, we try to train the model on a GPU (if available)
- by default, we get an overview of the missing sample/features in our training data (Missing Data Overview)
- by default, we use a PCA-based initialization strategy for the factors
- the model is automatically stored in the current working directory.

## Plot Results

After training, we can inspect all learned parameters, visualize the factors/weights, etc.


### ELBO

If the learningcurve flattens out, we know that our model converged.

In [14]:
plot_training_curve(model)

alt.Chart(...)

### Plot a heatmap of the weights

To perform downstream analysis, we need to extract the factor loadings and weights from the model. We can do this by calling the `get_factors()` and `get_weights()` method. This returns a dictionary with the weights for each modality and factors (for each group) respectively.


In [15]:
factors = model.get_factors()
weights = model.get_weights()

However, we also provide functionality to plot the weights directly.

### Factor Correlation Matrix

In [23]:
plot_factor_correlation(model)

alt.HConcatChart(...)

### Variance Explained Plot

In [24]:
plot_variance_explained(model)

alt.HConcatChart(...)

### Factor Values

In [25]:
plot_factor(model, factor=1)

alt.VConcatChart(...)

### Top weights per factor

In [11]:
plot_weights(model, "mutations", factor=1)

alt.LayerChart(...)

In [12]:
plot_weights(model, "mutations", factor=5)

alt.LayerChart(...)

In [13]:
plot_top_weights(model, view="mutations", factor=[1, 2, 3, 4, 5], orientation="horizontal")

TypeError: plot_top_weights() got an unexpected keyword argument 'view'

In [ ]:
import famo

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
model._cache["factors"]["group_1"].obs_names

In [ ]:
mdata.obs_names

In [ ]:
model._cache["factors"]["group_1"].obs = mdata.obs.loc[model._cache["factors"]["group_1"].obs_names, :].copy()
model._cache["factors"]["group_1"].obs["IGHV"] = model._cache["factors"]["group_1"].obs["IGHV"].astype(str).astype("category")
model._cache["factors"]["group_1"].obs["trisomy12"] = model._cache["factors"]["group_1"].obs["trisomy12"].astype(str).astype("category")
model._cache["factors"]["group_1"].obs.keys()

In [ ]:
model.factor_names

In [ ]:
famo.plotting.violinplot(model, factor_idx=1, groupby="IGHV")

In [ ]:
famo.plotting.violinplot(model, factor_idx=2, groupby="trisomy12")

In [ ]:
famo.plotting.violinplot(model, factor_idx=7, groupby="trisomy12")

In [ ]:
from famo.utils_downstream import test

In [ ]:
from famo import feature_sets as fs

In [ ]:
gene_sets = fs.from_gmt("c2.cp.reactome.v2023.2.Hs.symbols.gmt")
gene_sets

In [ ]:
gene_sets = gene_sets.filter(
    model.feature_names["mrna"], min_fraction=0.1, min_count=15
)
gene_sets

In [ ]:
gene_set_mask = gene_sets.to_mask(model.feature_names["mrna"])
gene_set_mask

In [ ]:
test_results_pos = test(model, "mrna", feature_sets=gene_set_mask, sign="pos")
test_results_neg = test(model, "mrna", feature_sets=gene_set_mask, sign="neg")

In [ ]:
test_df = test_results_pos["p_adj"].copy()

In [ ]:
for k in range(model.n_factors):
    print(model.factor_names[k])
    print(test_df.columns[test_df.iloc[k, :] < 0.05])

In [ ]:
import gseapy as gp
from gseapy import barplot

In [ ]:
relevant_factors = model.factor_names.tolist()[:5]

In [ ]:
model._cache["weights"]["mrna"].to_df().head()

In [ ]:
top = 200
# TODO: direction important if vanilla muvi
direction = "pos"
for factor_idx in relevant_factors:
    if factor_idx not in model.factor_names:
        continue
    gl = model._cache["weights"]["mrna"].to_df().loc[factor_idx, :].sort_values(ascending=False).index.tolist()
    if top is not None:
        gl = gl[:top]
    enr = gp.enrichr(
        gene_list=gl,
        gene_sets=[
            # "MSigDB_Hallmark_2020",
            # "KEGG_2021_Human",
            "Reactome_2022",
            # "GO_Biological_Process_2023",
        ],
        organism="human",
        outdir=None,
    )

    try:
        # categorical scatterplot
        ax = barplot(
            enr.results,
            column="Adjusted P-value",
            group="Gene_set",  # set group, so you could do a multi-sample/library comparsion
            size=10,
            top_term=5,
            figsize=(3, 5),
            color=["#1b9e77", "#d95f02", "#7570b3", "#e7298a"],  # set colors for group
            # color = {'MSigDB_Hallmark_2020':'blue', 'KEGG_2021_Human': 'salmon', 'Reactome_2022':'red'}
        )
        plt.title(f"{factor_idx} (top {top} loadings)")
        plt.show()
    except ValueError as e:
        print(e)